# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
df = pd.read_csv('/datasets/data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [3]:
df['education'].value_counts().sort_index()

ВЫСШЕЕ                   274
Высшее                   268
НАЧАЛЬНОЕ                 17
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
Начальное                 15
Неоконченное высшее       47
СРЕДНЕЕ                  772
Среднее                  711
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
высшее                  4718
начальное                250
неоконченное высшее      668
среднее                13750
ученая степень             4
Name: education, dtype: int64

In [4]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [5]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Вывод**

<div class="alert alert-info"> <b>Комментарий студента:</b>#подключил необходимые библитоеки и методом .info() вывел информацию о DataFrame ; # посчитал категории в столбце дети; # посчитал категории в столбце образование; #посчитал категории в столбце семейный статус
# OFF TOP: плохо разбираюсь пока в ЮпитерХаб и его навигации, изначальый вариант тоже был с хэштегами и выводами, не могу понять почему файл самый первый на первую проверку отправлялся без выводов и хэштегов!</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> а) методом info() увидел, что в DataFrame есть пустые значения б) в столбце children увидел отрицательные значения, странно что у некоторых семей по 20 детей, возможно это оишбки при вводе данных в)в столбце education, больше всего получили кредит люди имеющие среднее образование. В столбце проблемы с регистром надо перевести в нижний регистр г) в столбце family_status больше всего получили женатые. Так же нижний регистр необходим. д)в столбце purpose похожие цели по смыслу на получение кредита.</div>

## Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
df=df.fillna(0)

In [8]:
df.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [9]:
df = df[(df['children'] >= 0) & (df['children'] <= 20)]
df['children'].value_counts() 

0     14149
1      4818
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21478 non-null  int64  
 1   days_employed     21478 non-null  float64
 2   dob_years         21478 non-null  int64  
 3   education         21478 non-null  object 
 4   education_id      21478 non-null  int64  
 5   family_status     21478 non-null  object 
 6   family_status_id  21478 non-null  int64  
 7   gender            21478 non-null  object 
 8   income_type       21478 non-null  object 
 9   debt              21478 non-null  int64  
 10  total_income      21478 non-null  float64
 11  purpose           21478 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


**Вывод**
<div class="alert alert-info"> <b>Комментарий студента:</b> #проверил DataFrame на пустые значения; # заменил пустые значение на 0; # еще раз проверил DataFrame на пустые значения; # у детей надо убрать, отрицательные значения; # проверил, ушли отрицательные значения; # проверил на пустые значения<div>

<div class="alert alert-info"> <b>Комментарий студента:</b> а) Были пустые значения в столбцах: 1)days_employed, 2)total_income. б) Методом fillna() заполнил нулем все пустые значения </div>

### Замена типа данных

In [11]:
 df['education']=df['education'].str.lower()
df['family_status']=df['family_status'].str.lower()
df['family_status'].value_counts()

женат / замужем          12351
гражданский брак          4172
не женат / не замужем     2808
в разводе                 1191
вдовец / вдова             956
Name: family_status, dtype: int64

In [12]:
df['education'].value_counts()

среднее                15196
высшее                  5251
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [13]:
df['days_employed'] = df['days_employed'].astype(int)
df['total_income'] = df['total_income'].astype(int)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21478 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21478 non-null  int64 
 1   days_employed     21478 non-null  int64 
 2   dob_years         21478 non-null  int64 
 3   education         21478 non-null  object
 4   education_id      21478 non-null  int64 
 5   family_status     21478 non-null  object
 6   family_status_id  21478 non-null  int64 
 7   gender            21478 non-null  object
 8   income_type       21478 non-null  object
 9   debt              21478 non-null  int64 
 10  total_income      21478 non-null  int64 
 11  purpose           21478 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**
<div class="alert alert-info"> <b>Комментарий студента:</b> #нижний регистр; #нижний регистр; #методом astype(int) изменил тип данных на int ; #методом astype(int) изменил тип данных на int; #проверил изменение типа данных;</div>

<div class="alert alert-info"> <b>Комментарий студента:</b>а) перевел в нижний регистр столбец education. family_status б)методом astype(int) изменил у столбцов days_employed, total_income c float на int </div>

### Обработка дубликатов

In [15]:
df.duplicated().sum()

71

In [16]:
df = df.drop_duplicates().reset_index(drop=True)

In [17]:
df.duplicated().sum()

0

In [18]:
df['purpose'].value_counts().sort_index()

автомобили                                477
автомобиль                                492
высшее образование                        451
дополнительное образование                458
жилье                                     645
заняться высшим образованием              496
заняться образованием                     408
на покупку автомобиля                     470
на покупку подержанного автомобиля        477
на покупку своего автомобиля              504
на проведение свадьбы                     767
недвижимость                              632
образование                               444
операции с жильем                         651
операции с коммерческой недвижимостью     649
операции с недвижимостью                  673
операции со своей недвижимостью           624
покупка жилой недвижимости                604
покупка жилья                             643
покупка жилья для сдачи                   650
покупка жилья для семьи                   638
покупка коммерческой недвижимости 

**Вывод**
<div class="alert alert-info"> <b>Комментарий студента:</b>#проверил наличие дубликатов; #удалил дубликаты; #еще раз проверил; # в ручном поиске проверил наличие дубликатов в стоблбце purpose </div>

<div class="alert alert-info"> <b>Комментарий студента:</b>а) методом duplicates() нашел дубликаты в DataFrame б) Удалил дубликаты в) ручным поиском дубликатов нашел одинаковые по смыслу цели, их надо перевести к одной категории </div>

### Лемматизация

In [19]:
def purpose_lemma(row):
    lemmas = m.lemmatize(row)
    for word in lemmas:
        try:
            if 'свадьба' in word:
                return 'свадьба'
        
            if 'недвижимость' in word:
                return 'недвижимость'
        
            if 'жилье' in word:
                return 'недвижимость'
        
            if 'автомобиль' in word:
                return 'автомобиль'
        
            if 'образование' in word:
                return 'образование'
            
        except:
            print('Error in lemmatization')

In [20]:
df['grouped_purpose'] = df['purpose'].apply(purpose_lemma)

In [21]:
df['grouped_purpose'].value_counts()

недвижимость    10787
автомобиль       4295
образование      4003
свадьба          2322
Name: grouped_purpose, dtype: int64

**Вывод**
<div class="alert alert-info"> <b>Комментарий студента:</b> #добавил в толбце данные по категориям; # проверил категории </div>

<div class="alert alert-info"> <b>Комментарий студента:</b> а) определил цели по четырем категориям: свадьба, автомобиль недвижимость, образование </div>

### Категоризация данных

In [22]:
total_income_median = df['total_income'].median() 
total_income_median

135760.0

In [23]:
def group_total_income(total_income):
    if total_income < 37900:
        return 'низкий доход'
    if 37900 < total_income < total_income_median:
        return 'средний доход'
    if  total_income_median < total_income < 200000:
        return 'доход выше среднего'
    else:
        return 'высокий доход'

In [24]:
df['grouped_total_income'] = df['total_income'].apply(group_total_income)

In [25]:
df['grouped_total_income'].value_counts()

средний доход          8508
доход выше среднего    5645
высокий доход          5059
низкий доход           2195
Name: grouped_total_income, dtype: int64

In [26]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,grouped_purpose,grouped_total_income
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,высокий доход
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,доход выше среднего
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,доход выше среднего
5,0,-926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,высокий доход
6,0,-2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,высокий доход
7,0,-152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,доход выше среднего
8,2,-6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний доход
9,0,-2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,доход выше среднего


**Вывод** <div class="alert alert-info"> <b>Комментарий студента:</b> # в total_income_median назначил медиану total_income; #написал функцию для распределения total_income по категориям с ипользованием медианы, инфу зарплаты взял предположительную </div>

<div class="alert alert-info"> <b>Комментарий студента:</b> а) Медиану по столбцу total_income для использования категоризации по зарплатам б) group_total_income по столбцу total_income в) с помощью apply применил функцию к столбцу </div>

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [27]:
pd.pivot_table(df, index='children', values='debt')

,debt
children,
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


**Вывод**

<div class="alert alert-info"> <b>Комментарий студента:</b> Бездетные кредиторы чаще закрывают ккредиты , чем кредиторы с детьми. </div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
pd.pivot_table(df, index='children', values='debt')

,debt
children,
0,0.075438
1,0.092346
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


**Вывод**

<div class="alert alert-info"> <b>Комментарий студента:</b> Семейное положение влияет на вовремя погашеные кредиты. а) вдовец/вдова чаще вовремя погашают кредиты чем остальные б) следующие в разводе в) Чаще задолженность у не жинатых / не замужем </div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [29]:
pd.pivot_table(df,index='grouped_total_income', values='debt').sort_values(by='debt',ascending=True)

,debt
grouped_total_income,
высокий доход,0.070765
низкий доход,0.079727
средний доход,0.082393
доход выше среднего,0.089637


**Вывод**

<div class="alert alert-info"> <b>Комментарий студента:</b>Лучше кредитная история у людей с высоким доходом чем у остальных</div>

- Как разные цели кредита влияют на его возврат в срок?

In [30]:
pd.pivot_table(df, index='grouped_purpose', values='debt').sort_values(by='debt',ascending=True)

,debt
grouped_purpose,
недвижимость,0.072495
свадьба,0.080103
образование,0.092431
автомобиль,0.093597


**Вывод**

<div class="alert alert-info"> <b>Комментарий студента:</b> Кредиты на недвижимость чаще погашают чем на другие целиКредиты на недвижимость чаще погашают чем на другие цели </div>

## Шаг 4. Общий вывод

<div class="alert alert-info"> <b>Комментарий студента:</b> а)Семейное положение и наличие детей влияет на акт погашения в нужный срок б) В браке имеют меньше задолженности чем не женатые в)По наличию детей( у кого есть дети и у кого их нет) разница небольшая , она есть , но она небольшая </div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.